In [ ]:
from oot.oot import Oot
from oot.Accessor import Accessor
import os, random, requests, time
from oot.Utils import *

##### DEFINE SETTINGS
FORCE_ASSUMPTIONS = True
SEED = 'Deadpool2'
ROM = "ZOOTDEC.z64"
SELECTED_ENEMY = None #{'fn': 'En_Dodojr', 'object_fn': 'object_dodojr', 'var': ('0000', '')}
game = Oot(fn=ROM)
accessor = Accessor(game, force_assumptions=FORCE_ASSUMPTIONS, selected_enemy=SELECTED_ENEMY)
accessor.spawnAt(0x3)
#####

In [ ]:
# Cell for testing individual actor replacements
# Kokiri main spot04_room_0
# To replace rock guy old_actor=En_Ko, old_object_fn=object_km1
# Mido's House kokiri_home4_room_0
accessor.readData()
accessor.replaceActorInRoom('En_Test', 'En_Rr', 'Bmori1_room_6', var='FFFF')
#replaceActorInRoom('En_Box', 'En_Box', 'kokiri_home4_room_0', var='0700')
#replaceActorInRoom('En_Ko', 'En_Rr', 'spot04_room_0', var='0003', index=0, old_object_fn='object_os_anime')
#replaceActorInRoom('En_Karebaba', 'En_Skj', 'spot04_room_1', var='FFFF', old_object_fn='object_dekubaba')
# accessor.spawnAt(0x5)
accessor.writeData()

In [ ]:
# Creates a vanilla rom
accessor.readData()
rooms_list = accessor.reader.getRoomNames()
rooms = accessor.generateRooms(rooms_list)
accessor.spawnAt(0x3)
accessor.writeData()

In [3]:
from oot.oot import Oot
from oot.Accessor import Accessor
accessor = Accessor(Oot())
# Mido's House kokiri_home4_room_0
accessor.readData()
accessor.replaceActorInRoom('En_Ko', 'En_Rr', 'spot04_room_0', var='0003', index=0, old_object_fn='object_os_anime')
## 0x55 = Kokiri, 0x4 = Fire, 0x13 = Barinade (bdan_boss_room_1), 0x56 SFM
accessor.spawnAt(0x55)
#accessor.replaceActorInRoom('Boss_va', 'En_Box', 'bdan_boss_room_1', new_object_fn='object_box', var='0000', index=0, old_object_fn='object_bv')
accessor.writeData()

Using seed 1590507609
reading roms/ZOOTDEC.z64
reading roms/ZOOTDEC.z64
Writing roms/NEWZOOT.z64


In [5]:
rooms_list = accessor.reader.getRoomNames()
rooms = accessor.generateRooms(rooms_list)
print(rooms["ddan_room_5"].getInfo())

ddan_room_5 (Room File - Dodongo's Cavern) setup 0 @ 32956416
### Objects ###
0) 002B object_ddan_objects (Dodongo's Cavern Objects)
1) 018A object_bwall (Dodongo's Cavern Bombable Wall)
2) 0038 object_am (Armos)
3) 0031 object_bombf (Bomb Flower)
4) 000D object_firefly (Keese)
### Actors ###
0) 0054 En_Am (Armos) [FFFF] -> object_am
1) 0054 En_Am (Armos) [FFFF] -> object_am
2) 0054 En_Am (Armos) [FFFF] -> object_am
3) 0013 En_Firefly (Keese) [0000] -> object_firefly
4) 0013 En_Firefly (Keese) [0000] -> object_firefly
5) 004C En_Bombf (Bomb Flower) [FFFF] -> object_bombf
6) 004C En_Bombf (Bomb Flower) [FFFF] -> object_bombf
7) 004C En_Bombf (Bomb Flower) [FFFF] -> object_bombf
8) 004C En_Bombf (Bomb Flower) [FFFF] -> object_bombf
9) 0054 En_Am (Armos) [0000] -> object_am
10) 012A Obj_Switch (Global Dungeon Objects) [2200] -> gameplay_dangeon_keep


In [ ]:
# Running this cell should start to finish randomize the actors in the rom
actor_prefix = None # Only randomize among actors starting with this prefix
start_time = time.time()
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))

# Generate a dict with actor/{object: associated_object, variable: var} pairs
actors_associated = {}
for key, room in rooms.items():
    for num, setup in room.setups.items():
        for actor in setup['actors']:
            if actor.object_name is not 'NA':
                actors_associated[actor.filename] = {
                    "object": actor.object_name,
                    "variable": actor.var
                }
available_actors = {k:a for k,a in actors_associated.items() if k.startswith(actor_prefix)}
print("Actors Associated, {} sec".format(time.time()-start_time))

spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        key = '{}-{}'.format(r,n)
        room_spoiler = []
        spoilers[key] = room_spoiler
        for a, actor in enumerate(setup['actors']):
            if actor.filename in available_actors and actor.filename not in [rm["new"]["filename"] for rm in room_spoiler]:
                new_actor_name = random.choice(list(available_actors.keys()))
                old_actor_info = {"filename": actor.filename, "object": actor.object_name, "description": actor.description, "variable": actor.var}
                room.replaceActor(actor.filename, new_actor_name, new_var=available_actors[new_actor_name]['variable'])
                new_actor_info = {"filename": setup['actors'][a].filename, "object": setup['actors'][a].object_name, "description": setup['actors'][a].description, "variable": actor.var}
                room_spoiler.append({"old": old_actor_info, "new": new_actor_info})
        spoilers[r] = room_spoiler
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Print out spoilers
writeSpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))

In [ ]:
import os
f=os.path.join("roms","oozdec.z64")
os.path.isfile(f)